In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:

import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
from keras import backend as K
import tensorflow as tf
import datetime
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Root directory of the project
ROOT_DIR = os.path.abspath("")

# Import Mask RCNN
sys.path.append(ROOT_DIR+"/codigo")  # To find local version of the library
from funcionesAux import utils, visualize, metricas, obtenerMascaras, io, training
import funcionesAux.model as modellib

In [ ]:
#Guardar datos de entrenamiento procesados
training.etiquetarYguardarDatos("samplesTesting/procesadosZgz")

In [ ]:
#Cargar datos de entrenamiento procesados
train_data1,train_data2,train_label,test_data1,test_data2,test_label=training.cargarDatos("samplesTesting/procesadosZgz")

In [6]:
# Clear any logs from previous runs
#!rm -rf ./logs/ 

In [ ]:
import keras 
from keras import backend as K
from keras.models import Sequential,Model
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Input, concatenate, Activation, Subtract

from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.losses import binary_crossentropy


#(14,14,256)->(1,1,256)
def bloqueReduccion(in1,in2):
    x1=Conv2D(256, (5, 5), strides=(2,2))
    x2=BatchNormalization()
    x3=Activation('relu')
    x4=Conv2D(256, (5, 5))
    x5=BatchNormalization()
    x6=Activation('relu')
    
    #objeto 1
    ob1=x1(in1)
    ob1=x2(ob1)
    ob1=x3(ob1)
    ob1=x4(ob1)
    ob1=x5(ob1)
    ob1=x6(ob1)
    
    #objeto 2
    ob2=x1(in2)
    ob2=x2(ob2)
    ob2=x3(ob2)
    ob2=x4(ob2)
    ob2=x5(ob2)
    ob2=x6(ob2)
    
    return ob1,ob2

def createModel():

    in1 = Input(shape=(14,14,256))
    in2 = Input(shape=(14,14,256))

    #(1,1,256)->(256)
    reduccion_in1,reduccion_in2 = bloqueReduccion(in1,in2)
    reduccion_in1 = Flatten()(reduccion_in1)
    reduccion_in2 = Flatten()(reduccion_in2)

    #(reduccion_in1-reduccion_in2)
    z = Subtract()([reduccion_in1, reduccion_in2])

    z=Dense(256, activation='relu')(z)
    z=Dense(128, activation='relu')(z)
    z=Dense(64, activation='relu')(z)
    z=Dense(1, activation='sigmoid')(z)

    modelRAM = Model(inputs=[in1,in2], outputs=[z])

    modelRAM.compile(Adam(lr=.0001), loss='binary_crossentropy', metrics=['accuracy'])
    
    return modelRAM


In [ ]:
#PINTAR GRAFICAS para ver como se entrena la red en cada epoca
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)

# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "redes/1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

# Create a new model instance
modelRAM=createModel()

modelRAM.summary() 

# Save the weights using the `checkpoint_path` format
modelRAM.save_weights(checkpoint_path.format(epoch=0))

modelRAM.fit([train_data1,train_data2], train_label, callbacks=[tensorboard_callback,cp_callback], batch_size=10, validation_split=0.2, epochs=200)


In [ ]:
#Comprobar la accuracy de la red para cada epoca y la epoca para la cual la accuracy es mayor
model = createModel()

test=[]
numFichero=0
maxAcc=0
maxIndex=0
for i in range(201):
    # Load the previously saved weights
    model.load_weights("redes/1/cp-"+str(numFichero).zfill(4)+".ckpt")

    loss,acc = model.evaluate([test_data1,test_data2], test_label, verbose=0)
    #print("test loss, test acc:", loss,acc)
    test.append(acc)
    if acc>maxAcc:
        maxAcc=acc
        maxIndex=i
    
    numFichero+=1

plt.plot(test)
print('acc->',maxAcc)
print('epoca->',maxIndex)


In [ ]:
#Comprobar la accuracy para una epoca en concreto
model = createModel()
model.load_weights("redes/1/cp-0136.ckpt")
loss,acc = model.evaluate([test_data1,test_data2], test_label, verbose=0)
print(acc)


In [ ]:
#Mostrar accuracy y loss de cada epoca
%tensorboard --logdir logs/fit

In [ ]:
!kill 25363

In [ ]:
#Mostrar accuracy y loss del reentrenamiento de Mask R-CNN
%tensorboard --logdir cars